In [ ]:
from nltk.stem.porter import *
from pathlib import Path
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score
POS_DICT = []
BIO_DICT = []
PATH_DICT = []

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def index_list(dict, key):
    if key in dict:
        return dict.index(key)
    else:
        dict.append(key)
        return dict.index(key)

In [ ]:
def read_data(pth):
    pairs_data = []
    sentence = []
    with open(pth, 'r') as file:
        lines_in = file.read().split('\n')
        for line in lines_in:
            if line == '':
                if sentence == '' or len(sentence) < 3:
                    sentence = []
                    continue
                # print("s", sentence)
                pairs_data += find_pair(sentence)
                sentence = []
                continue
            else:
                sentence.append(line)
    # print("pairs_data", pairs_data)
    if len(sentence)>0 : 
      pairs_data += find_pair(sentence)
    print("pair_data size path ",pth, len(pairs_data))
    return pairs_data

In [ ]:
def find_pair(sentence):
    pairs_data = []
    whole_POS_path = []
    whole_BIO_path = [] # ------TODO: introduce BIO path-------
    for i, word in enumerate(sentence):
        data = word.split('\t')
        whole_POS_path.append(data[1])
        whole_BIO_path.append(data[2])
        if len(data) < 6:
            continue
        if data[5] == "ARG1":
            ARG_POS = data[1]
            ARG_BIO = data[2]
            ARG_num = data[3]

        elif data[5] == "PRED":
            PRED_POS = data[1]
            PRED_BIO = data[2]
            PRED_num = data[3]
            PRED_index = i

    for i, word in enumerate(sentence):
        data = word.split('\t')
        if i == PRED_index:
            continue
        if i < PRED_index:
            current_path = [whole_POS_path[x] for x in range(i, PRED_index+1)]
        else:
            current_path = [whole_POS_path[x] for x in range(PRED_index, i+1)]
        current_path = ','.join(current_path)
        if len(data) == 6 and data[5] == "ARG1":
            role = 1
        else:
            role = 0
        pairs_data.append([index_list(POS_DICT, PRED_POS)] + [index_list(BIO_DICT, PRED_BIO)]
                          + [index_list(POS_DICT, data[1])] + [index_list(BIO_DICT, data[2])]
                          + [int(PRED_num) - int(data[3])] + [index_list(PATH_DICT, current_path)]
                          + [role])
    return pairs_data

In [ ]:
if __name__ == "__main__":


    data_dir = Path("/content/drive/MyDrive/NLP 2022/nombank_train_dev_test/percentage")
    data_dir2 = Path("/content/drive/MyDrive/NLP 2022/Results")
    train_data_path = data_dir / 'train.data'
    valid_data_path = data_dir / 'dev.data'
    test_data_path = data_dir / 'test.data'
    output_path = data_dir2 / 'RF_PPB.txt'
    
    pair_train = np.array(read_data(train_data_path))# loaded by function "read_data"
    x = pair_train[:, 0:-1]
    y = pair_train[:, -1]

    # pair_test = onehot_pair(np.load("dev.npy"), withpth=0)
    pair_test = np.array(read_data(test_data_path))
    x_test = pair_test[:, 0:-1]
    y_test = pair_test[:, -1]

    print("training DT")
    clf = RandomForestClassifier(n_estimators=200, criterion="gini", min_samples_split=10, max_features="auto", n_jobs=4)  
    clf.fit(x, y)  
    y_pred_DT = clf.predict(x_test) 
    print("confusion_matrix", confusion_matrix(y_test, y_pred_DT))
    print("accuracy_score", accuracy_score(y_test,y_pred_DT))

    N = 0
    
    with open(test_data_path, 'r') as file_in:
        with open(output_path, 'w') as file_out:
            lines_in = file_in.read().split('\n')
            #print("total no of lines is ", test_data_path,len(lines_in))
            #C=0
            for line in lines_in:
                #print("processing line number : ",C)
                data = line.split("\t")
                if line == '' or len(line) <3:
                    file_out.write("\n")
                    continue
                if len(data) >= 6 and data[5] == "PRED":
                    file_out.write(line + "\n")
                    continue
                if len(data) < 6:
                    data.append("")
                if y_pred_DT[N] == 1:
                    data[5] = "ARG1"
                    N+=1
                else:
                    data[5] = ""
                    N+=1
                
                file_out.write("\t".join(data) + "\n")
            
            print(N)
    assert N == len(y_pred_DT)

pair_data size path  /content/drive/MyDrive/NLP 2022/nombank_train_dev_test/percentage/train.data 59555
pair_data size path  /content/drive/MyDrive/NLP 2022/nombank_train_dev_test/percentage/test.data 4128
training DT
confusion_matrix [[3932   45]
 [  99   52]]
accuracy_score 0.9651162790697675
4128
